# CRM Dashboard 2025 - Data Upload and Cleaning

This notebook handles:
- Mounting Google Drive
- Manual CSV selection from /CRM_Dashboard_Exports/
- Data cleaning and merging
- Derived field generation
- Export to combined_crm_leads.csv

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os
from pathlib import Path

# List available CSVs
drive_folder = '/content/drive/MyDrive/CRM_Dashboard_Exports/'
csv_files = [f for f in os.listdir(drive_folder) if f.endswith('.csv')]

print('Available CSV files:')
for i, file in enumerate(csv_files, 1):
    print(f'{i}. {file}')

In [ ]:
# Manual selection
selection = input('Enter file numbers to process (comma-separated, e.g., 1,3,5): ')
selected_indices = [int(x.strip()) - 1 for x in selection.split(',')]
selected_files = [csv_files[i] for i in selected_indices]

print('Selected files:')
for f in selected_files:
    print(f'  - {f}')

In [ ]:
# Load and merge selected CSVs
dfs = []
for file in selected_files:
    df_temp = pd.read_csv(os.path.join(drive_folder, file))
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)
print(f'Total rows loaded: {len(df)}')

In [ ]:
# Data cleaning
# Replace blanks with N/A
df = df.fillna('N/A')

# Convert date fields
df["Date Added"] = pd.to_datetime(df["Date Added"], errors='coerce')
df["Last Assigned"] = pd.to_datetime(df["Last Assigned"], errors='coerce')

# Drop duplicates
df = df.drop_duplicates()

print(f'Rows after cleaning: {len(df)}')

In [ ]:
# Derive Project column
def derive_project(row):
    if row.get("Interested: Vesper") == "Yes":
        return "Vesper"
    elif row.get("Interested: The Code") == "Yes":
        return "The Code"
    elif row.get("Interested: Belvedere") == "Yes":
        return "Belvedere"
    else:
        return "Unspecified"

df["Project"] = df.apply(derive_project, axis=1)
print('Project column created')

In [ ]:
# Derive Lead Type column
df["Lead Type"] = df.apply(
    lambda x: "Reengaged"
    if pd.notnull(x["Last Assigned"]) and x["Last Assigned"] > x["Date Added"]
    else "New",
    axis=1
)
print('Lead Type column created')

In [ ]:
# Save to data folder
output_path = '/content/drive/MyDrive/CRM_Dashboard_Exports/data/combined_crm_leads.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False)

print(f'Saved to {output_path}')
print(f'Final dataset: {len(df)} rows, {len(df.columns)} columns')